<a href="https://colab.research.google.com/github/jaimeclr/3D/blob/main/circulos_en_area_to_stl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from trimesh.creation import cylinder
from trimesh import Trimesh
from trimesh.util import concatenate

# Parámetros de entrada (definidos en un solo lugar)
base = 98    # mm
altura = 60  # mm
diametro = 21.5 # mm # Diámetro del círculo grande para el cálculo de disposición
diametro_pequeno = 11 # mm # Diámetro del círculo pequeño (agujero)
espesor = 10 # mm # Grosor en 3D

def calcular_disposicion(base, altura, diametro):
    """
    Calcula la disposición hexagonal de círculos sin separación dentro de un rectángulo
    """
    radio = diametro / 2
    # The distance between the centers of adjacent circles is the diameter (no separation)
    distancia_centros = diametro
    paso_x = distancia_centros
    paso_y = (math.sqrt(3) / 2) * distancia_centros

    centros = []
    i = 0
    while True:
        y = i * paso_y + radio # Start the first row at radio distance from the bottom edge
        if y + radio > altura:
            break

        j = 0
        while True:
            if i % 2 == 0:
                x = j * paso_x + radio # Start the first circle at radio distance from the left edge
            else:
                x = j * paso_x + paso_x / 2 + radio # Start the shifted row at radio distance plus half step_x

            if x + radio > base:
                break

            # Ensure the entire circle is within the bounds
            if x - radio >= 0 and x + radio <= base and y - radio >= 0 and y + radio <= altura:
                 centros.append((x, y))

            j += 1
        i += 1

    # Calculate approximate rows and columns for reporting
    filas = i
    # This is an approximation for reporting, based on the number of circles in the first row
    columnas = len([c for c in centros if abs(c[1] - centros[0][1]) < 1e-9])


    return centros, filas, columnas

# --- Main Execution ---

# Calcular disposición de los cilindros
disposicion = 'hexagonal sin separación'
centros, filas, columnas = calcular_disposicion(base, altura, diametro)

print(f"Resultados para la disposición {disposicion}:")
print(f"Cantidad de círculos: {len(centros)}")
print(f"Filas aproximadas: {filas}, Columnas aproximadas: {columnas}")

# Crear la geometría de los cilindros
radio_agujero_pequeno = diametro_pequeno / 2
cilindros_geometria = []

for centro in centros:
    x, y = centro
    cilindro = cylinder(
        radius=radio_agujero_pequeno,
        height=espesor,
        transform=np.array([
            [1, 0, 0, x],
            [0, 1, 0, y],
            [0, 0, 1, espesor / 2],
            [0, 0, 0, 1]
        ])
    )
    cilindros_geometria.append(cilindro)

# Combinar las geometrías de los cilindros
if cilindros_geometria:
    cilindros_combinados = concatenate(cilindros_geometria)

    # Exportar a archivo stl
    file_path = 'cilindros_combinados.stl'
    cilindros_combinados.export(file_path)

    print(f"Combined cylinder geometry successfully exported to {file_path}")
else:
    print("No cylinders were generated, skipping export.")


# Confirmar la exportación (assuming cylinders were generated and exported)
if 'cilindros_combinados' in locals():
     print("The STL file named 'cilindros_combinados.stl' containing the solid cylinders has been successfully created and saved.")

Resultados para la disposición hexagonal sin separación:
Cantidad de círculos: 12
Filas aproximadas: 3, Columnas aproximadas: 4
Combined cylinder geometry successfully exported to cilindros_combinados.stl
The STL file named 'cilindros_combinados.stl' containing the solid cylinders has been successfully created and saved.
